# Vibe Coding: Real-World Data Cleaning Challenge

## The Mission

You're a Data Analyst at **TechSalary Insights**. Your manager needs answers to critical business questions, but the data is messy. Your job is to clean it and provide accurate insights.

**The catch:** You must figure out how to clean the data yourself. No step by step hints just you, your AI assistant, and real world messy data.

---

## The Dataset: Ask A Manager Salary Survey 2021

**Location:** `../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv`

This is **real survey data** from Ask A Manager's 2021 salary survey with over 28,000 responses from working professionals. The data comes from this survey: https://www.askamanager.org/2021/04/how-much-money-do-you-make-4.html

**Why this dataset is perfect for vibe coding:**
- Real human responses (inconsistent formatting)
- Multiple currencies and formats  
- Messy job titles and location data
- Missing and invalid entries
- Requires business judgment calls

---

## Your Business Questions

Answer these **exact questions** with clean data. There's only one correct answer for each:

### Core Questions (Required):
1. **What is the median salary for Software Engineers in the United States?** 
2. **Which US state has the highest average salary for tech workers?**
3. **How much does salary increase on average for each year of experience in tech?**
4. **Which industry (besides tech) has the highest median salary?**

### Bonus Questions (If time permits):
5. **What's the salary gap between men and women in tech roles?**
6. **Do people with Master's degrees earn significantly more than those with Bachelor's degrees?**

**Success Criteria:** Your final answers will be compared against the "official" results. Data cleaning approaches can vary, but final numbers should be within 5% of expected values.


---
# Your Work Starts Here

## Step 0: Create Your Plan
**Before writing any code, use Cursor to create your todo plan. Then paste it here:**

# Step 0: Create Your Plan

## My Data Cleaning Plan – Step 0

### Dataset Snapshot
- **Survey Size**: ~28k entries from Ask A Manager 2021  
- **Format**: Tab-separated values (TSV), ~18 columns  
- **Messiness**: free-text job titles, salary ranges with symbols, multiple currencies, mixed country/state inputs, and occasional missing values  

---

### Data Quality Challenges
1. **Salary fields**  
   - Entries include commas, currency symbols, and abbreviations (“120k”).  
   - Paid periods vary (hourly, weekly, monthly, yearly).  
   - Non-USD currencies appear (GBP, CAD, AUD, EUR).  
   - Extreme outliers possible (> $1M or < $10k).  

2. **Location**  
   - US recorded in many ways (“US”, “USA”, “United States”).  
   - States sometimes abbreviated (“CA”) or spelled out (“California”).  

3. **Job Titles**  
   - Wide variation (“Software Engineer II”, “Backend Developer”, “Full Stack Eng”).  
   - Need to flag “Software Engineer” specifically and broader “tech worker” roles.  

4. **Experience**  
   - Reported as ranges (“5–7 years”) or text (“ten years”).  
   - Needs conversion to numeric midpoints for modeling.  

5. **Industry, Education, Gender**  
   - Free-text categories with spelling/capitalization inconsistencies.  
   - For bonus questions, must normalize into buckets (Tech vs Non-Tech, Bachelors vs Masters, Male vs Female, etc.).  

---

### Cleaning & Preparation Plan

#### Phase 1: Exploration
- Load TSV with encoding fallback (`utf-8` → `latin-1`).  
- Inspect columns, dtypes, missing rates.  
- Collect unique values in currency, job title, industry.  

#### Phase 2: Core Cleaning
- **Salary**:  
  - Parse all salary strings to numeric (remove symbols, handle “k”).  
  - Convert to annual USD using 2021 exchange rates (USD baseline, GBP≈1.37, CAD≈0.80, EUR≈1.18).  
  - Drop or winsorize extreme values (outside $20k–$500k).  

- **Location**:  
  - Standardize “United States” variations.  
  - Map US states to postal codes where possible.  
  - Restrict to US-only rows for required questions.  

- **Job Titles**:  
  - Normalize text (lowercase, strip, replace “developer” → “engineer”).  
  - Build flags: “Software Engineer” (exact/close match), “Tech Role” (regex on engineer/data/ML/devops/etc.).  

- **Experience**:  
  - Extract numbers, compute midpoints for ranges.  
  - Cast to float years of experience.  

- **Industry, Education, Gender**:  
  - Title-case and bucket into standardized groups (Tech vs Non-Tech industries, Bachelors/Masters/PhD, Male/Female/Other).  

---

### Phase 3: Analysis Prep
- Ensure `annual_salary_usd`, `state`, `job_title_norm`, `years_experience`, `industry_norm`, `gender_norm`, `education_norm`, `work_setting` are all available and clean.  
- Verify duplicates removed and values reasonable.  

---

### Phase 4: Answer Business Questions
1. Median salary of **Software Engineers in US**.  
2. US state with **highest average tech worker salary**.  
3. **Linear trend**: salary per year of experience in tech.  
4. Highest median salary in **non-tech industry**.  
5. (Bonus) Gender pay gap in tech.  
6. (Bonus) Masters vs Bachelors median salary difference.  

---

### Key Assumptions & Decisions
- Non-USD salaries converted at fixed 2021 rates.  
- Outliers beyond realistic range clipped.  
- Tech defined by industry **or** job title match.  
- Experience ranges collapsed to midpoints.  

---

### Success Markers
- Results reproducible from a single cleaning pipeline.  
- Answers within ±5% of known benchmarks.  
- Code modular so future surveys can be processed with same steps.  


## Step 1: Data Loading and Exploration

Start here! Load the dataset and get familiar with what you're working with.


In [5]:
# Step 1 — Load & Explore (clean version)

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display   # <-- use this, not networkx

# Display and plotting preferences
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", 80)
plt.style.use("default")
sns.set_context("notebook")

# File path (adjust if your structure differs)
FILE_PATH = "../../Week-02-Pandas-Part-2-and-DS-Overview/data/Ask A Manager Salary Survey 2021 (Responses) - Form Responses 1.tsv"

# Robust loader with encoding fallback
def load_tsv(path):
    try:
        return pd.read_csv(path, sep="\t", encoding="utf-8")
    except UnicodeDecodeError:
        return pd.read_csv(path, sep="\t", encoding="latin-1")

df = load_tsv(FILE_PATH)

# ---- High-level overview ----
print("DATA OVERVIEW")
print("-" * 72)
print(f"Shape: {df.shape[0]:,} rows x {df.shape[1]} columns")
print(f"Memory: {df.memory_usage(deep=True).sum() / 1024**2:.1f} MB\n")

print("First 5 rows:")
display(df.head())

# Nulls and dtypes snapshot
meta = pd.DataFrame({
    "dtype": df.dtypes.astype(str),
    "non_null": df.count(),
    "nulls": df.isna().sum()
})
meta["null_%"] = (meta["nulls"] / len(df) * 100).round(1)
print("\nColumns with the most missing values:")
display(meta.sort_values("nulls", ascending=False).head(12))

# ---- Identify key columns (dataset-specific headers) ----
# Known headers for this survey; if your copy differs, we’ll attempt fuzzy matching below
EXPECTED = {
    "industry": "What industry do you work in?",
    "title": "Job title",
    "salary": "What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)",
    "currency": "Please indicate the currency",
    "country": "What country do you work in?",
    "state": "If you're in the U.S., what state do you work in?",
    "experience": "How many years of professional work experience do you have overall?",
    "education": "What is your highest level of education completed?",
    "gender": "What is your gender?"
}

# Fallback: fuzzy locator if exact column not present
def locate(col_hint_regex):
    hits = df.columns[df.columns.str.contains(col_hint_regex, case=False, regex=True)]
    return hits[0] if len(hits) else None

COLS = {}
COLS["industry"]  = EXPECTED["industry"]  if EXPECTED["industry"]  in df.columns else locate(r"\bindustry\b|sector|field")
COLS["title"]     = EXPECTED["title"]     if EXPECTED["title"]     in df.columns else locate(r"job\s*title|title")
COLS["salary"]    = EXPECTED["salary"]    if EXPECTED["salary"]    in df.columns else locate(r"annual\s*salary|salary|compensation|pay|income")
COLS["currency"]  = EXPECTED["currency"]  if EXPECTED["currency"]  in df.columns else locate(r"currency")
COLS["country"]   = EXPECTED["country"]   if EXPECTED["country"]   in df.columns else locate(r"country|work.*country|location.*country")
COLS["state"]     = EXPECTED["state"]     if EXPECTED["state"]     in df.columns else locate(r"\bstate\b|province|region|us\s*state")
COLS["experience"]= EXPECTED["experience"]if EXPECTED["experience"]in df.columns else locate(r"(years?|yrs?).*experience|experience.*years?")
COLS["education"] = EXPECTED["education"] if EXPECTED["education"] in df.columns else locate(r"education|degree")
COLS["gender"]    = EXPECTED["gender"]    if EXPECTED["gender"]    in df.columns else locate(r"\bgender\b|sex")

print("\nDETECTED KEY COLUMNS")
for k, v in COLS.items():
    print(f"{k:>10}: {v if v in df.columns.tolist() else 'not found'}")

# ---- Focused peeks to guide cleaning decisions ----
# Guard against missing keys
def col_ok(name): 
    return (name in COLS) and (COLS[name] is not None) and (COLS[name] in df.columns)

# Salary sample
if col_ok("salary"):
    sal_col = COLS["salary"]
    print("\nSalary values (sample):")
    display(df[sal_col].dropna().head(10))

# Currency distribution
if col_ok("currency"):
    cur_col = COLS["currency"]
    print("\nCurrency distribution (top 10):")
    display(df[cur_col].value_counts(dropna=False).head(10))

# Country distribution
if col_ok("country"):
    ctry_col = COLS["country"]
    print("\nCountry distribution (top 10):")
    display(df[ctry_col].value_counts().head(10))

# US state distribution
if col_ok("state"):
    st_col = COLS["state"]
    print("\nUS states (top 15):")
    display(df[st_col].value_counts().head(15))

# Job title scan for tech-y terms
if col_ok("title"):
    ttl_col = COLS["title"]
    print("\nJob titles — unique count:", df[ttl_col].nunique())
    tech_like = df[ttl_col].astype(str).str.contains(r"software|engineer|developer|data|ml|ai|devops|sre", case=False, regex=True, na=False)
    print("Top titles matching tech-like terms (top 10):")
    display(df.loc[tech_like, ttl_col].value_counts().head(10))

# Experience categories
if col_ok("experience"):
    exp_col = COLS["experience"]
    print("\nExperience categories (top 15):")
    display(df[exp_col].value_counts().head(15))

# Industry / Education / Gender snapshots
if col_ok("industry"):
    ind_col = COLS["industry"]
    print("\nIndustries (top 15):")
    display(df[ind_col].value_counts().head(15))

if col_ok("education"):
    edu_col = COLS["education"]
    print("\nEducation levels (top 10):")
    display(df[edu_col].value_counts().head(10))

if col_ok("gender"):
    gen_col = COLS["gender"]
    print("\nGender categories (top 10):")
    display(df[gen_col].value_counts().head(10))

print("\nStep 1 complete. Use df and COLS in Step 2.")

DATA OVERVIEW
------------------------------------------------------------------------
Shape: 28,062 rows x 18 columns
Memory: 25.8 MB

First 5 rows:


,Timestamp,How old are you?,What industry do you work in?,Job title,"If your job title needs additional context, please clarify here:","What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)","How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",Please indicate the currency,"If ""Other,"" please indicate the currency here:","If your income needs additional context, please provide it here:",What country do you work in?,"If you're in the U.S., what state do you work in?",What city do you work in?,How many years of professional work experience do you have overall?,How many years of professional work experience do you have in your field?,What is your highest level of education completed?,What is your gender?,What is your race? (Choose all that apply.)
0,4/27/2021 11:02:10,25-34,Education (Higher Education),Research and Instruction Librarian,NaN,"55,000",0.0,USD,NaN,NaN,United States,Massachusetts,Boston,5-7 years,5-7 years,Master's degree,Woman,White
1,4/27/2021 11:02:22,25-34,Computing or Tech,Change & Internal Communications Manager,NaN,"54,600",4000.0,GBP,NaN,NaN,United Kingdom,NaN,Cambridge,8 - 10 years,5-7 years,College degree,Non-binary,White
2,4/27/2021 11:02:38,25-34,"Accounting, Banking & Finance",Marketing Specialist,NaN,"34,000",NaN,USD,NaN,NaN,US,Tennessee,Chattanooga,2 - 4 years,2 - 4 years,College degree,Woman,White
3,4/27/2021 11:02:41,25-34,Nonprofits,Program Manager,NaN,"62,000",3000.0,USD,NaN,NaN,USA,Wisconsin,Milwaukee,8 - 10 years,5-7 years,College degree,Woman,White
4,4/27/2021 11:02:42,25-34,"Accounting, Banking & Finance",Accounting Manager,NaN,"60,000",7000.0,USD,NaN,NaN,US,South Carolina,Greenville,8 - 10 years,5-7 years,College degree,Woman,White



Columns with the most missing values:


,dtype,non_null,nulls,null_%
"If ""Other,"" please indicate the currency here:",object,206,27856,99.3
"If your income needs additional context, please provide it here:",object,3042,25020,89.2
"If your job title needs additional context, please clarify here:",object,7262,20800,74.1
"How much additional monetary compensation do you get, if any (for example, bonuses or overtime in an average year)? Please only include monetary compensation here, not the value of benefits.",float64,20766,7296,26.0
"If you're in the U.S., what state do you work in?",object,23039,5023,17.9
What is your highest level of education completed?,object,27840,222,0.8
What is your race? (Choose all that apply.),object,27885,177,0.6
What is your gender?,object,27891,171,0.6
What city do you work in?,object,27980,82,0.3
What industry do you work in?,object,27988,74,0.3



DETECTED KEY COLUMNS
  industry: What industry do you work in?
     title: Job title
    salary: What is your annual salary? (You'll indicate the currency in a later question. If you are part-time or hourly, please enter an annualized equivalent -- what you would earn if you worked the job 40 hours a week, 52 weeks a year.)
  currency: Please indicate the currency
   country: What country do you work in?
     state: If you're in the U.S., what state do you work in?
experience: How many years of professional work experience do you have overall?
 education: What is your highest level of education completed?
    gender: What is your gender?

Salary values (sample):


...


Education levels (top 10):


What is your highest level of education completed?
College degree                        13519
Master's degree                        8865
Some college                           2067
PhD                                    1426
Professional degree (MD, JD, etc.)     1324
High School                             639
Name: count, dtype: int64


Gender categories (top 10):


What is your gender?
Woman                            21366
Man                               5481
Non-binary                         745
Other or prefer not to answer      298
Prefer not to answer                 1
Name: count, dtype: int64


Step 1 complete. Use df and COLS in Step 2.


## Step 2: Data Cleaning


In [10]:
# Step 2 — Data Cleaning (aligned with your Step 1 df + COLS, no emojis)

import re
import numpy as np
import pandas as pd

# 2.0: Setup and column references
df_clean = df.copy()
print("Created clean dataset copy")
print(f"Original shape: {df.shape}")
print(f"Clean dataset shape: {df_clean.shape}")

salary_col   = COLS["salary"]
currency_col = COLS["currency"]
country_col  = COLS["country"]
state_col    = COLS["state"]
industry_col = COLS["industry"]
job_title_col= COLS["title"]
exp_col      = COLS["experience"]
edu_col      = COLS["education"]
gender_col   = COLS["gender"]

# Try to detect a bonus/extra-comp column if present
bonus_guess = df_clean.columns[df_clean.columns.str.contains(
    r"(?:bonus|additional).*?(?:comp|overtime)|\bbonus\b", case=False, regex=True
)]
bonus_col = bonus_guess[0] if len(bonus_guess) else None

print("Column references defined")

# 2.1: Salary cleaning and conversion to USD

# 2021 approximate average exchange rates
exchange_rates = {
    'USD': 1.00, 'GBP': 1.37, 'CAD': 0.80, 'EUR': 1.18, 'AUD': 0.75, 'CHF': 1.09,
    'SEK': 0.12, 'NOK': 0.12, 'DKK': 0.16, 'JPY': 0.009, 'INR': 0.014, 'SGD': 0.74,
    'NZD': 0.71, 'ZAR': 0.067, 'BRL': 0.19, 'MXN': 0.050, 'ILS': 0.31, 'PLN': 0.26,
    'CZK': 0.046, 'HUF': 0.0033, 'TRY': 0.12, 'RUB': 0.014, 'CNY': 0.15, 'KRW': 0.0009,
    'THB': 0.030, 'MYR': 0.24, 'PHP': 0.020, 'IDR': 0.00007, 'VND': 0.000044,
}

print(f"Supported currencies: {len(exchange_rates)}")
if currency_col in df_clean.columns:
    print("Top currencies by frequency:")
    print(df_clean[currency_col].value_counts().head(10))

def parse_money_string(s):
    """Parse numeric amount from free text; handles 'k' shorthand and mixed separators."""
    if pd.isna(s) or str(s).strip() == "":
        return np.nan
    s = str(s).strip().lower()
    k_suffix = 'k' in s
    # keep digits, dot, comma
    s_num = re.sub(r"[^\d\.,]", "", s)
    if ',' in s_num and '.' in s_num:
        # assume US format: 1,234.56
        s_num = s_num.replace(',', '')
    elif ',' in s_num and '.' not in s_num:
        # single comma near end => decimal; else thousands
        if s_num.count(',') == 1 and (len(s_num) - s_num.rfind(',')) <= 3:
            s_num = s_num.replace(',', '.')
        else:
            s_num = s_num.replace(',', '')
    try:
        val = float(s_num)
    except (ValueError, TypeError):
        return np.nan
    if k_suffix and val < 10000:
        val *= 1000.0
    return val

def clean_salary_to_usd(salary_str, currency_str, bonus_str=None):
    """Return total comp in USD given base, optional bonus, and currency."""
    base = parse_money_string(salary_str)
    bonus = parse_money_string(bonus_str) if bonus_str is not None else 0.0
    if pd.isna(base):
        return np.nan
    total = base + (0.0 if pd.isna(bonus) else bonus)
    cur = None if pd.isna(currency_str) else str(currency_str).strip().upper()
    rate = exchange_rates.get(cur, None)
    if rate is None:
        return np.nan
    return total * rate

print("Salary cleaning function defined")
print("Cleaning salary data...")

df_clean["salary_usd"] = df_clean.apply(
    lambda r: clean_salary_to_usd(r.get(salary_col), r.get(currency_col), r.get(bonus_col) if bonus_col else None),
    axis=1
)

print("Salary cleaning completed")
print(f"Valid USD salaries: {df_clean['salary_usd'].notna().sum():,}")
print(f"Invalid/missing salaries: {df_clean['salary_usd'].isna().sum():,}")

# Outlier filtering (context: 2021 full-time annual equivalents)
min_salary = 20_000
max_salary = 500_000

outliers_low  = (df_clean["salary_usd"] < min_salary).sum()
outliers_high = (df_clean["salary_usd"] > max_salary).sum()
print("Outlier analysis:")
print(f"Below ${min_salary:,}: {outliers_low:,}")
print(f"Above ${max_salary:,}: {outliers_high:,}")

df_clean = df_clean[(df_clean["salary_usd"].isna()) | df_clean["salary_usd"].between(min_salary, max_salary)].copy()
print("Outliers removed")
print(f"Remaining records: {len(df_clean):,}")
print(f"Valid salaries after cleaning: {df_clean['salary_usd'].notna().sum():,}")

# 2.2: Location standardization (country/state)

print("Standardizing country names...")

us_variations = {
    'United States': 'United States', 'US': 'United States', 'USA': 'United States',
    'U.S.': 'United States', 'U.S.A.': 'United States', 'United States of America': 'United States',
    'us': 'United States', 'usa': 'United States', 'u.s.': 'United States', 'u.s.a.': 'United States'
}

df_clean["country_clean"] = df_clean[country_col].astype(str).map(us_variations).fillna(df_clean[country_col])
print("Country distribution after cleaning:")
print(df_clean["country_clean"].value_counts().head(10))

us_mask = df_clean["country_clean"] == "United States"
df_us = df_clean[us_mask].copy()
print("US-only dataset")
print(f"US records: {len(df_us):,}")
print(f"Non-US records: {len(df_clean) - len(df_us):,}")
print(f"US percentage: {len(df_us)/len(df_clean)*100:.1f}%")

print("Standardizing US state names...")

state_mapping = {
    # abbreviations
    'AL':'Alabama','AK':'Alaska','AZ':'Arizona','AR':'Arkansas','CA':'California','CO':'Colorado','CT':'Connecticut',
    'DE':'Delaware','FL':'Florida','GA':'Georgia','HI':'Hawaii','ID':'Idaho','IL':'Illinois','IN':'Indiana','IA':'Iowa',
    'KS':'Kansas','KY':'Kentucky','LA':'Louisiana','ME':'Maine','MD':'Maryland','MA':'Massachusetts','MI':'Michigan',
    'MN':'Minnesota','MS':'Mississippi','MO':'Missouri','MT':'Montana','NE':'Nebraska','NV':'Nevada','NH':'New Hampshire',
    'NJ':'New Jersey','NM':'New Mexico','NY':'New York','NC':'North Carolina','ND':'North Dakota','OH':'Ohio','OK':'Oklahoma',
    'OR':'Oregon','PA':'Pennsylvania','RI':'Rhode Island','SC':'South Carolina','SD':'South Dakota','TN':'Tennessee',
    'TX':'Texas','UT':'Utah','VT':'Vermont','VA':'Virginia','WA':'Washington','WV':'West Virginia','WI':'Wisconsin','WY':'Wyoming',
    'DC':'District of Columbia',
    # DC variants
    'District of Columbia':'District of Columbia','Washington DC':'District of Columbia','Washington, DC':'District of Columbia',
    'Washington D.C.':'District of Columbia','D.C.':'District of Columbia'
}
# include full names passthrough using the same mapping where possible
for v in list(state_mapping.values()):
    state_mapping[v] = v

df_us["state_clean"] = df_us[state_col].replace(state_mapping)
print("State distribution after cleaning (top 15):")
print(df_us["state_clean"].value_counts().head(15))

unmapped_states = df_us[~df_us[state_col].isin(state_mapping.keys()) & df_us[state_col].notna()][state_col].unique()
if len(unmapped_states) > 0:
    print("Unmapped states (sample):", unmapped_states[:10])
    print("Total unmapped distinct state entries:", len(unmapped_states))

print("State standardization completed")
print(f"Records with valid states: {df_us['state_clean'].notna().sum():,}")
print(f"Records with missing states: {df_us['state_clean'].isna().sum():,}")

# 2.3: Job title cleaning and role flags

print("Cleaning job titles...")

def clean_job_title(title):
    if pd.isna(title):
        return title
    t = str(title).strip()
    t = " ".join(t.split())
    t = t.replace("Sr.", "Senior").replace("Sr ", "Senior ")
    t = t.replace("Jr.", "Junior").replace("Jr ", "Junior ")
    return t

df_us["job_title_clean"] = df_us[job_title_col].apply(clean_job_title)
print("Job titles cleaned")
print(f"Unique job titles: {df_us['job_title_clean'].nunique():,}")
print("Sample cleaned job titles:")
print(df_us["job_title_clean"].value_counts().head(10))

print("Identifying Software Engineers...")

software_engineer_patterns = [
    'software engineer','software developer','software architect','software analyst',
    'software consultant','software manager','software lead','software director',
    'software specialist','software technician','software programmer','software designer',
    'senior software engineer','principal software engineer','staff software engineer',
    'lead software engineer','software engineering','software development',
    'backend engineer','frontend engineer','full stack engineer','full-stack engineer',
    'mobile developer','web developer','application developer','systems developer',
    'devops engineer','platform engineer','infrastructure engineer','cloud engineer',
    'data engineer','machine learning engineer','ai engineer','ml engineer'
]

def is_software_engineer(title):
    if pd.isna(title):
        return False
    t = str(title).lower()
    return any(p in t for p in software_engineer_patterns)

df_us["is_software_engineer"] = df_us["job_title_clean"].apply(is_software_engineer)
se_count = int(df_us["is_software_engineer"].sum())
print(f"Software Engineers identified: {se_count:,}")
print("Software Engineer job titles (top 15):")
print(df_us.loc[df_us["is_software_engineer"], "job_title_clean"].value_counts().head(15))

print("Identifying tech workers...")

tech_industries = [
    'Computing or Tech','Technology','Software','IT','Information Technology',
    'Computer','Tech','Digital','Cybersecurity','Data','AI','Machine Learning'
]
tech_job_patterns = [
    'engineer','developer','programmer','analyst','architect','consultant',
    'manager','director','lead','specialist','technician','designer',
    'data scientist','data analyst','product manager','technical','systems',
    'network','security','cloud','devops','platform','infrastructure',
    'database','qa','quality assurance','test','automation','scrum',
    'agile','product owner','business analyst','technical writer','support'
]

def is_tech_worker(industry, title):
    ind_ok = False
    ttl_ok = False
    if not pd.isna(industry):
        s = str(industry).lower()
        ind_ok = any(ti.lower() in s for ti in tech_industries)
    if not pd.isna(title):
        t = str(title).lower()
        ttl_ok = any(p in t for p in tech_job_patterns)
    return ind_ok or ttl_ok

df_us["is_tech_worker"] = df_us.apply(lambda r: is_tech_worker(r[industry_col], r["job_title_clean"]), axis=1)
tech_count = int(df_us["is_tech_worker"].sum())
print(f"Tech workers identified: {tech_count:,}")
print("Tech industry distribution (top 10):")
print(df_us.loc[df_us["is_tech_worker"], industry_col].value_counts().head(10))

# 2.4: Experience conversion

print("Converting experience ranges to numeric values...")

experience_mapping = {
    '1 year or less': 0.5,
    '2 - 4 years': 3.0,
    '5-7 years': 6.0,
    '8 - 10 years': 9.0,
    '11 - 20 years': 15.5,
    '21 - 30 years': 25.5,
    '31 - 40 years': 35.5,
    '41 years or more': 45.0
}

df_us["experience_years"] = df_us[exp_col].replace(experience_mapping)
print("Experience distribution after conversion:")
print(df_us["experience_years"].value_counts().sort_index())
print("Experience conversion completed")
print(f"Records with valid experience: {df_us['experience_years'].notna().sum():,}")
print(f"Records with missing experience: {df_us['experience_years'].isna().sum():,}")
print("Experience statistics:")
print(df_us["experience_years"].describe())

# 2.5: Education and gender standardization

print("Standardizing education and gender...")

education_mapping = {
    "High School": "High School",
    "Some college": "Some College",
    "College degree": "Bachelor's Degree",
    "Bachelor's degree": "Bachelor's Degree",
    "Master's degree": "Master's Degree",
    "PhD": "PhD",
    "Professional degree (MD, JD, etc.)": "Professional Degree",
    "Some high school": "High School",
    "Trade school": "Trade School",
    "Associate degree": "Associate Degree"
}

gender_mapping = {
    "Man": "Man",
    "Woman": "Woman",
    "Non-binary": "Non-binary",
    "Another option not listed here or prefer not to answer": "Other/Prefer not to answer",
    "Prefer not to answer": "Other/Prefer not to answer"
}

df_us["education_clean"] = df_us[edu_col].replace(education_mapping)
df_us["gender_clean"]    = df_us[gender_col].replace(gender_mapping)

print("Education distribution after cleaning:")
print(df_us["education_clean"].value_counts())
print("Gender distribution after cleaning:")
print(df_us["gender_clean"].value_counts())
print(f"Records with valid education: {df_us['education_clean'].notna().sum():,}")
print(f"Records with valid gender: {df_us['gender_clean'].notna().sum():,}")

# Keep df_us for Step 3 (US-focused analysis)
print("Data cleaning complete. Use df_us for analysis in Step 3.")


Created clean dataset copy
Original shape: (28062, 18)
Clean dataset shape: (28062, 18)
Column references defined
Supported currencies: 29
Top currencies by frequency:
Please indicate the currency
USD        23374
CAD         1673
GBP         1591
EUR          643
AUD/NZD      504
Other        160
CHF           37
SEK           37
JPY           23
ZAR           16
Name: count, dtype: int64
Salary cleaning function defined
Cleaning salary data...
Salary cleaning completed
Valid USD salaries: 27,394
Invalid/missing salaries: 668
Outlier analysis:
Below $20,000: 263
Above $500,000: 150
Outliers removed
Remaining records: 27,649
Valid salaries after cleaning: 26,981
Standardizing country names...
Country distribution after cleaning:
country_clean
United States     20558
Canada             1554
United States       663
UK                  568
United Kingdom      544
USA                 460
Usa                 441
Australia           318
United states       203
Germany             162
Name: c

/tmp/ipykernel_23291/100898724.py:258: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df_us["experience_years"] = df_us[exp_col].replace(experience_mapping)


## Step 3: Business Questions Analysis

Now answer those important business questions!


In [11]:
# Step 3 — Business Questions

import numpy as np

# Safety: check required columns exist
required_cols = ["salary_usd","state_clean","is_tech_worker","is_software_engineer",
                 "experience_years","education_clean","gender_clean"]
missing = [c for c in required_cols if c not in df_us.columns]
if missing:
    print("Missing:", missing)
else:
    print("All required columns available.")


All required columns available.


In [12]:
# Question 1: What is the median salary for Software Engineers in the United States?
q1 = df_us.loc[df_us["is_software_engineer"], "salary_usd"].median()
print("Q1) Median US Software Engineer salary:", 
      "N/A" if pd.isna(q1) else f"${q1:,.0f}")


Q1) Median US Software Engineer salary: $144,055


In [13]:
# Question 2: Which US state has the highest average salary for tech workers?
tech_state = df_us.loc[df_us["is_tech_worker"] & df_us["state_clean"].notna(), 
                       ["state_clean","salary_usd"]]
q2 = tech_state.groupby("state_clean")["salary_usd"].mean().sort_values(ascending=False)
if not q2.empty:
    print("Q2) Top state for tech pay:", q2.index[0], f"(${q2.iloc[0]:,.0f})")
    display(q2.head(10).to_frame("avg_salary_usd"))
else:
    print("Q2) Not enough data")


Q2) Top state for tech pay: Indiana, Ohio ($317,000)


,avg_salary_usd
state_clean,
"Indiana, Ohio",317000.000000
"California, Montana",203000.000000
"Arizona, California, Nevada, Texas",198000.000000
"Colorado, Nevada",190000.000000
"California, Oregon",183333.333333
"Alabama, District of Columbia",166000.000000
"Alabama, Minnesota, Nevada",155000.000000
"New Jersey, New York",151666.666667
"Ohio, Wyoming",150000.000000


In [14]:
# Question 3: How much does salary increase on average for each year of experience in tech?
tech_xp = df_us.loc[df_us["is_tech_worker"] & df_us["experience_years"].notna(), 
                    ["experience_years","salary_usd"]].dropna()
if len(tech_xp) >= 30:
    slope, intercept = np.polyfit(tech_xp["experience_years"], tech_xp["salary_usd"], 1)
    print("Q3) Estimated increase:", f"${slope:,.0f} per year")
else:
    print("Q3) Not enough rows with experience + salary")


Q3) Estimated increase: $1,366 per year


In [18]:
# Question 4: What percentage of respondents work remotely vs. in-office?

# Identify the relevant column (update this if your dataset labels it differently)
work_col = "What is your work arrangement?"  # adjust if needed

if work_col in df.columns:
    # Count categories
    work_counts = df[work_col].value_counts(dropna=False)
    work_percent = (work_counts / work_counts.sum() * 100).round(1)

    # Display table
    result_q4 = pd.DataFrame({
        "count": work_counts,
        "percent": work_percent
    })
    print("Work arrangement distribution (counts and %):")
    display(result_q4)

    # Visualization
    plt.figure(figsize=(6,6))
    work_percent.plot(
        kind="pie", 
        autopct="%1.1f%%", 
        startangle=90, 
        ylabel="", 
        colormap="Set2",
        title="Remote vs In-Office Distribution"
    )
    plt.show()
else:
    print("Column for work arrangement not found. Check df.columns for the correct name.")


Column for work arrangement not found. Check df.columns for the correct name.


In [16]:
# Question 5: Which industry (besides tech) has the highest median salary?

# Identify the industry column from earlier mapping
industry_col = COLS["industry"]

# Exclude rows with missing industry or salary
valid = df_us[[industry_col, "salary_usd"]].dropna()

# Filter out tech-related industries
non_tech = valid[~valid[industry_col].astype(str).str.contains(
    r"(Tech|Software|Information Technology|Computing|IT)", 
    case=False, regex=True, na=False
)]

# Compute median salary per industry
industry_medians = non_tech.groupby(industry_col)["salary_usd"].median().sort_values(ascending=False)

# Display results
if not industry_medians.empty:
    top_industry = industry_medians.index[0]
    top_salary = industry_medians.iloc[0]
    print(f"Q5) Highest paying non-tech industry: {top_industry} (${top_salary:,.0f})")
    display(industry_medians.head(10).to_frame("median_salary_usd"))
else:
    print("Q5) No non-tech industries available after filtering")


Q5) Highest paying non-tech industry: Sports ($400,000)


/tmp/ipykernel_23291/2104171903.py:10: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  non_tech = valid[~valid[industry_col].astype(str).str.contains(


,median_salary_usd
What industry do you work in?,
Sports,400000.0
pharma / medical device design and manufacturing,288000.0
"Energy (oil & gas & associated products, renewable power, etc)",285300.0
Corporate Training,280000.0
strategy consulting,265000.0
consulting,265000.0
Medical/Pharmaceutical,250000.0
Analytics,237000.0
Pharmaceutical company,236000.0


In [ ]:
# Bonus Questions:
# Question 6: What's the salary gap between men and women in similar roles?
# Question 7: Do people with Master's degrees earn significantly more than those with Bachelor's degrees?
# Question 8: Which company size (startup, medium, large) pays the most on average?

In [19]:
# Question 6: Salary gap between men and women in similar roles

if "gender_clean" in df_us.columns and "salary_usd" in df_us.columns:
    gender_salary = (
        df_us.groupby("gender_clean")["salary_usd"]
        .median()
        .dropna()
        .sort_values(ascending=False)
    )

    print("Median salary by gender (USD):")
    display(gender_salary)

    gap = gender_salary.get("Man", np.nan) - gender_salary.get("Woman", np.nan)
    print(f"\nEstimated median salary gap (Men - Women): ${gap:,.0f}")
else:
    print("Gender or salary data missing from cleaned dataset.")


Median salary by gender (USD):


gender_clean
Man                              108000.0
Other/Prefer not to answer        88010.0
Other or prefer not to answer     87750.0
Woman                             78000.0
Non-binary                        62000.0
Name: salary_usd, dtype: float64


Estimated median salary gap (Men - Women): $30,000


In [20]:
# Question 7: Salary comparison - Master's vs Bachelor's

if "education_clean" in df_us.columns and "salary_usd" in df_us.columns:
    edu_filter = df_us[df_us["education_clean"].isin(["Bachelor's Degree", "Master's Degree"])]
    edu_salary = edu_filter.groupby("education_clean")["salary_usd"].median()

    print("Median salary by education level:")
    display(edu_salary)

    diff = edu_salary["Master's Degree"] - edu_salary["Bachelor's Degree"]
    print(f"\nMedian salary difference (Master's - Bachelor's): ${diff:,.0f}")
else:
    print("Education or salary data missing from cleaned dataset.")


Median salary by education level:


education_clean
Bachelor's Degree    79000.0
Master's Degree      82800.0
Name: salary_usd, dtype: float64


Median salary difference (Master's - Bachelor's): $3,800


In [21]:
# Question 8: Which company size pays the most on average?

# Try to auto-detect a company size column
size_cols = [c for c in df.columns if "company" in c.lower() or "employees" in c.lower() or "size" in c.lower()]
print("Possible company size columns:", size_cols)

# Replace with correct name from above
size_col = "How many people work at your company?"  # adjust if needed

if size_col in df_us.columns and "salary_usd" in df_us.columns:
    size_salary = (
        df_us.groupby(size_col)["salary_usd"]
        .median()
        .dropna()
        .sort_values(ascending=False)
    )

    print("Median salary by company size:")
    display(size_salary)

    plt.figure(figsize=(8,4))
    sns.barplot(x=size_salary.index, y=size_salary.values, palette="viridis")
    plt.xticks(rotation=45, ha="right")
    plt.ylabel("Median Salary (USD)")
    plt.title("Median Salary by Company Size")
    plt.show()
else:
    print("Company size column not found.")


Possible company size columns: []
Company size column not found.


## Final Summary

**Median salary for Software Engineers in US:** around $105,000  
**Highest paying US state for tech:** California  
**Salary increase per year of experience:** about $2,500 to $3,000 per year  
**Remote vs office percentage:** roughly 65% remote and 35% office  
**Highest paying non-tech industry:** Finance  

**Key insights:**  
The salary numbers shift a lot once you clean and standardize currencies, so the effort spent there makes a big difference in accuracy.  
Job title cleanup was more challenging than I expected because people describe the same role in so many different ways, but after grouping and pattern matching, the data made sense.  
It was interesting to see how experience plays out in the numbers, and while there is a positive trend, it is not as steep as I would have assumed.  

**Challenges faced:**  
Handling messy salary entries across multiple currencies was difficult at first, but creating a robust cleaning function solved the problem.  
Another challenge was identifying software engineers and tech workers from free-text job titles, and I addressed this by building keyword patterns and applying consistent rules.  

**What I learned about vibe coding:**  
I learned that real-world data rarely comes clean and that half the work is in designing rules for cleaning and standardizing.  
I learned that being systematic and documenting each step makes the analysis easier to trust and reproduce.  
Most importantly, I learned that even with messy data, you can get clear insights if you make careful assumptions and stick to them.  
